In [ ]:
import sys
import os

os.chdir("..")
os.chdir("./src")
# sys.path.append("./src")

In [ ]:
from IPython import display
from IPython.display import clear_output
import pylab as pl
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, jit
from time import time

# from helpers import *
from LDMIBSS import *

# np.random.seed(250)
%load_ext autoreload
%autoreload 2

In [ ]:
N = 10000
NumberofSources = 5
NumberofMixtures = 8
S = generate_correlated_copula_sources(
    rho=0.7,
    df=4,
    n_sources=NumberofSources,
    size_sources=N,
    decreasing_correlation=False,
)

print("The following is the correlation matrix of sources")
display_matrix(np.corrcoef(S))

# Generate Mxr random mixing from i.i.d N(0,1)
A = np.random.randn(NumberofMixtures, NumberofSources)
X = np.dot(A, S)

SNR = 30
X, NoisePart = addWGN(X, SNR, return_noise=True)

SNRinp = 10 * np.log10(
    np.sum(np.mean((X - NoisePart) ** 2, axis=1))
    / np.sum(np.mean(NoisePart**2, axis=1))
)
print("The following is the mixture matrix A")
display_matrix(A)
print("Input SNR is : {}".format(SNRinp))

In [ ]:
s_dim = S.shape[0]
x_dim = X.shape[0]
debug_iteration_point = 200
model = LDMIBSS(s_dim=s_dim, x_dim=x_dim, set_ground_truth=True, S=S, A=A)

In [ ]:
model.fit_batch_nnantisparse(
    X,
    epsilon=1e-5,
    mu_start=100,
    n_iterations=10000,
    method="correlation",
    debug_iteration_point=debug_iteration_point,
    plot_in_jupyter=True,
)

In [ ]:
mpl.rcParams["xtick.labelsize"] = 18
mpl.rcParams["ytick.labelsize"] = 18
plot_convergence_plot(
    model.SINR_list,
    xlabel="Number of Iterations / {}".format(debug_iteration_point),
    ylabel="SINR (dB)",
    title="SINR Convergence Plot",
    colorcode=None,
    linewidth=1.8,
)

print("Final SINR: {}".format(np.array(model.SINR_list[-1])))

In [ ]:
W = model.W
Y = W @ X
10 * np.log10(CalculateSINR(Y, S)[0])

In [ ]:
Y = W @ X
print(Y.shape, X.shape, S.shape)
Y_ = signed_and_permutation_corrected_sources(S.T, Y.T)
coef_ = (Y_ * S.T).sum(axis=0) / (Y_ * Y_).sum(axis=0)
Y_ = coef_ * Y_
snr(S.T, Y_)

In [ ]:
Y = W @ X
Y_ = signed_and_permutation_corrected_sources(S.T, Y.T).T
subplot_1D_signals(
    X=Y_.T[0:100],
    title="Extracted Signals (Sign and Permutation Corrected)",
    figsize=(15.2, 9),
    colorcode=None,
)
subplot_1D_signals(
    X=S.T[0:100], title="Original Signals", figsize=(15.2, 9), colorcode=None
)